In [1]:
import tensorflow as tf
import datetime
import numpy as np

#### implemento un modello con il dataset MNIST

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(64, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform()),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 64)                50240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

#### path per i log di TensorBoard

In [3]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

#### Imposto un learning rate che muta in base all'epoca di addestramento

In [5]:
file_writer = tf.summary.create_file_writer(log_dir)
file_writer.set_as_default()

def lr_schedule(epoch):
    learning_rate = 0.003
    if epoch > 10:
        learning_rate = 0.002
    if epoch > 20:
        learning_rate = 0.001
    if epoch > 40:
        learning_rate = 0.0005

    # log del learning-rate in TensorBoard
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

#### Addestro il modello

In [ ]:
model.fit(x=x_train,
          y=y_train,
          epochs=50,
          batch_size=4096,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback, lr_callback])

Epoch 1/50
15/15 [==============================] - 3s 92ms/step - loss: 2.4756 - accuracy: 0.1010 - val_loss: 2.2975 - val_accuracy: 0.1425 - lr: 0.0030
Epoch 2/50
15/15 [==============================] - 1s 72ms/step - loss: 2.3216 - accuracy: 0.1130 - val_loss: 2.2636 - val_accuracy: 0.1806 - lr: 0.0030
Epoch 3/50
15/15 [==============================] - 1s 75ms/step - loss: 2.2008 - accuracy: 0.1921 - val_loss: 1.9663 - val_accuracy: 0.3038 - lr: 0.0030
Epoch 4/50
15/15 [==============================] - 1s 77ms/step - loss: 1.7884 - accuracy: 0.3224 - val_loss: 1.3703 - val_accuracy: 0.5372 - lr: 0.0030
Epoch 5/50
15/15 [==============================] - 1s 74ms/step - loss: 1.3766 - accuracy: 0.4622 - val_loss: 1.0500 - val_accuracy: 0.6808 - lr: 0.0030
Epoch 6/50
15/15 [==============================] - 1s 72ms/step - loss: 1.1485 - accuracy: 0.5729 - val_loss: 0.8425 - val_accuracy: 0.7632 - lr: 0.0030
Epoch 7/50
15/15 [==============================] - 1s 69ms/step - loss: 0.9

#### Scrittura di un log utilizzabile da TensorBoard delle prime 